# Single Play Poker Enviroment

This Notebooks contains enviroment which plays Poker with single person.

## install require packages

In [4]:
!pip install gym

    100% |████████████████████████████████| 1.5MB 19.0MB/s 
    100% |████████████████████████████████| 1.0MB 29.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/cf/a5/c9/87967963aa32540d543e51bcf0d0fc19c5d68b8f49598d3b98
Successfully built gym
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Class of Enviroment

### imports

In [4]:
from enum import Enum
import random
import gym
import numpy as np
from io import StringIO
import sys
from typing import Tuple

### Suit Class(Enum)

* this class defines Card's suit. Clover or Diamond or Heart or Spade.

In [5]:
class Suit(Enum):
    Clover    = 0
    Diamond   = 1
    Heart     = 2
    Spade     = 3

### Helper variable that converts Suit -> String or number -> String

In [6]:
suit_to_str = {
    Suit.Heart:   "Heart",
    Suit.Spade:   "Spade",
    Suit.Clover:  "Clover",
    Suit.Diamond: "Diamond"
}

num_to_string = [0, "Ace", "Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Nine", "Ten", "Jack", "Queen", "King"]

### Card class

* this class implements one card

In [7]:
class Card:
    def __init__(self, suit: Suit, num: int):
        self.suit = suit  # type: Suit
        self.num  = num   # type: int
    
    def __repr__(self) -> str:
        return "{} of {}".format(num_to_string[self.num], suit_to_str[self.suit])
    
    def __eq__(self, other:'Card') -> bool:
        return self.num == other.num and self.suit == other.suit
    
    def __lt__(self, other:'Card') -> bool:
        if self.num == other.num:
            return self.suit.value < other.suit.value
        else:
            return self.num < other.num
    
    def __le__(self, other:'Card') -> bool:
        if self.num == other.num:
            return self.suit.value <= other.suit.value
        else:
            return self.num <= other.num 
        
    def __gt__(self, other:'Card') -> bool:
        if self.num == other.num:
            return self.suit.value > other.suit.value
        else:
            return self.num > other.num
        
    def __ge__(self, other:'Card') -> bool:
        if self.num == other.num:
            return self.suit.value >= other.suit.value
        else:
            return self.num >= other.num
    
    def __hash__(self) -> str:
        return hash((self.suit.value, self.num))
    
    def clone(self) -> 'Card':
        return Card(self.suit, self.num)
    
    def get_serial(self) -> int:
        return self.suit.value * 13 + self.num

### Player class

* this class implements player status.

In [8]:
class Player:
    def __init__(self, initial_coin:int=1000, bid_amount:int=10):
        self.initial_coin = initial_coin # type: int
        self.hand         = [] # type: List[Card]
        self.bid_amount   = bid_amount
        self.reset()
        
    def reset(self) -> None:
        self.coin = self.initial_coin
        self.hand.clear()

    def draw_card(self, card:Card) -> None:
        self.hand.append(card.clone())
        self.hand.sort()
    
    def bid(self) -> int:
        self.coin -= self.bid_amount
        return self.bid_amount

### Poker Action Class

- this class(Enum) shows player's actions.

In [9]:
class TexusHoldemActions(Enum):
    DRAW   = 0
    DOUBLE = 1
    CALL   = 2
    FOLD   = 3
    CHECK  = 4

### UserAction class

* this class implements User took action

In [10]:
class UserAction:
    def __init__(self, action:TexusHoldemActions, drop_cards:[int]=None):
        self.action     = action # type: SinglePlayPokerActions
        self.drop_cards = drop_cards # type: [int]

### SinglePlayPorkerStates class

In [11]:
class TexusHoldemStates(Enum):
    BEFORE_PLAYABLE = 0
    PLAYABLE        = 1
    DONE_EPISODE    = 2

### Poker class

* this class shows poker and helper method to judge poker for your hand.

In [58]:
class Poker(Enum):
    RoyalStraightFlash = 0
    StraightFlash      = 1
    FourCard           = 2
    FullHouse          = 3
    Flash              = 4
    Straight           = 5
    ThreeCard          = 6
    TwoPair            = 7
    OnePair            = 8
    Nothing            = 9
    
    @classmethod
    def createCardMap(cls, cards:[Card]):
        card_map = [[],[],[],[]]
        for i in range(4):
            card_map[i] = [0] * 14
        
        for card in cards:
            card_map[card.suit.value][card.num] += 1

        return np.array(card_map)
    
    @classmethod
    def getPoker(cls, hand:[Card], public:[Card]) -> 'Poker':
        all_cards = sorted(hand + public)
        card_map = cls.createCardMap(all_cards)
        for judgement in (Poker.isRoyalStraightFlash, Poker.isStraightFlash, Poker.isFourCard,
                          Poker.isFullHouse, Poker.isFlash, Poker.isStraight, Poker.isThreeCard,
                          Poker.isNPair):
            judge = judgement(card_map)
            if judge is not Poker.Nothing:
                return judge
        return Poker.Nothing
    
    @classmethod
    def isRoyalStraightFlash(cls, card_map) -> 'Poker':
        possibility = [0,1,0,0,0,0,0,0,0,0,1,1,1,1]
        if np.count_nonzero(np.dot(card_map, possibility)==5) == 1:
            return cls.RoyalStraightFlash
        return cls.Nothing
    
    @classmethod
    def isStraightFlash(cls, card_map) -> 'Poker':
        for line in card_map:
            for i in range(1, 10):
                if np.all(line[i:i+5]):
                    return cls.StraightFlash
        return cls.Nothing

    @classmethod
    def isFourCard(cls, card_map) -> 'Poker':
        if True in np.all(card_map == 1, axis = 0):
                return cls.FourCard
        return cls.Nothing

    @classmethod
    def isFullHouse(cls, card_map) -> 'Poker':
        hist = np.sum(card_map, axis=0)
        pair = np.count_nonzero(hist == 2)
        triple = np.count_nonzero(hist == 3)
        if pair >= 1 and triple >= 1 or triple >= 2:
            return cls.FullHouse
        return cls.Nothing
    
    @classmethod
    def isFlash(cls, card_map) -> 'Poker':
        hist = np.sum(card_map, axis=1)
        if np.count_nonzero(hist == 5) > 0:
            return Poker.Flash
        return cls.Nothing
    
    @classmethod
    def isStraight(cls, card_map) -> 'Poker':
        exists = np.where(np.sum(card_map, axis=0) > 0, 1, 0)
        for i in range(1, 10):
            if np.all(exists[i:i+5]):
                return cls.Straight
        return cls.Nothing
    
    @classmethod
    def isThreeCard(cls, card_map) -> 'Poker':
        hist = np.sum(card_map, axis=0)
        pair = np.count_nonzero(hist == 2)
        triple = np.count_nonzero(hist == 3)
        if np.count_nonzero(hist >= 3) >= 1:
            return cls.ThreeCard
        return cls.Nothing
    
    @classmethod
    def isNPair(cls, card_map) -> 'Poker':
        hist = np.sum(card_map, axis=0)
        pair = np.count_nonzero(hist >= 2)
        return [cls.Nothing, cls.OnePair, cls.TwoPair][pair]

### UserState class

* this class implements UserState, such as amount of bid in this game and current hand.

In [11]:
class UserState:
    def __init__(self, current_bid:int, current_hand:int):
        self.current_bid = current_bid
        self.current_hand = []
        for card in current_hand:
            self.current_hand.append(card.clone())
    
    def __hash__(self) -> str:
        return hash((self.current_bid, self.current_hand[0], self.current_hand[1], self.current_hand[2], self.current_hand[3], self.current_hand[4]))
    
    def __repr__(self) -> str:
        return (self.current_bid, self.current_hand).__repr__()

### SimplePokerEnv

* You hardly reached this class.
* this class shows Environment OpenAI gym.

In [12]:
class SimplePokerEnv(gym.Env):
    
    reward_rate_map = {
        Poker.RoyalStraightFlash: 100,
        Poker.StraightFlash:       25,
        Poker.FourCard:            15,
        Poker.FullHouse:            7,
        Poker.Flash:                5,
        Poker.Straight:             4,
        Poker.ThreeCard:            3,
        Poker.TwoPair:              2,
        Poker.OnePair:              1,
        Poker.Nothing:             -1
    }

    def __init__(self, default_bid:int=10):
        super().__init__()
        self.action_space   = gym.spaces.Discrete(31)
        self.decks          = [] # type: List[Card]
        self.cards_in_place = []
        self.default_bid    = default_bid # type: int
        self.player         = Player() # type: Player
        self.states         = [
            SinglePlayPokerStates.PLAYABLE,
            SinglePlayPokerStates.DONE_EPISODE
        ] # type: List[SinglePlayPokerStates]
        self.reward_range   = [  
            SimplePokerEnv.reward_rate_map[Poker.Nothing] * self.max_draw,
            SimplePokerEnv.reward_rate_map[Poker.RoyalStraightFlash] * self.max_draw
        ]
        self.actions        = [
            SinglePlayPokerActions.DRAW,
            SinglePlayPokerActions.DOUBLE
        ]
        self._reset()

    def _reset(self) -> UserState:
        self.draw_count   = 0 # type: int
        self.state        = self.states[0] # type: SinglePlayPokerStates
        self.player_coin  = self.initial_coin # type: int
        self.card_pointer = 0
        self.decks.clear()
        for suit in (Suit.Heart, Suit.Spade, Suit.Clover, Suit.Diamond):
            for i in range(1, 14):
                self.decks.append(Card(suit, i))
        random.shuffle(self.decks)
        
        self.player.reset()
        for i in range(5):
            self.player.draw_card(self.decks[i])
        #self.player.hand.sort()
        self.card_pointer = 5
        self.bids = self.player.bid()
        return self._observe()
    
    def _observe(self) -> UserState:
        clone_hand = []
        for card in self.player.hand:
            clone_hand.append(card.clone())
        return UserState(self.bids, clone_hand)
    
    def _step(self, action: UserAction) -> Tuple[UserState, int, bool, dict]:
        reward = 0 # type: int
        observation = None # type: UserState
        done = False #type: bool
        info = {} #type: dict
        if self.state == SinglePlayPokerStates.PLAYABLE:
            if action.action == SinglePlayPokerActions.DOUBLE:
                bid = self.player.bid()
                self.bids += bid
                if self.draw_count >= self.max_draw:
                    #print("You cannot draw anymore(max drawable count is {})".format(self.max_draw))
                    self.state = SinglePlayPokerStates.DONE_EPISODE
            if not self.state == SinglePlayPokerStates.DONE_EPISODE:
                n = action.drop_cards.count(1)
                if n == 0:
                    self.state = SinglePlayPokerStates.DONE_EPISODE
                else:
                    for i in reversed(range(5)):
                        if action.drop_cards[i] == 1:
                            self.player.hand.remove(self.player.hand[i])
                    for i in range(n):
                        self.player.draw_card(self.decks[self.card_pointer+i])
                    self.card_pointer += n

                    self.draw_count += 1
            # print("You draw {} times.".format(self.draw_count))
            if self.draw_count >= self.max_draw:
                #print("You cannot draw anymore(max drawable count is {})".format(self.max_draw))
                self.state = SinglePlayPokerStates.DONE_EPISODE

                    
        if self.state == SinglePlayPokerStates.DONE_EPISODE:
            poker, reward = self._get_reward()
            done = True
            info["poker"] = poker
        observation = self._observe()
        return observation, reward, done, info

    def _get_reward(self) -> Tuple[Poker, int]:
        poker = Poker.Nothing
        #self.player.hand.sort()

        for judgement in (Poker.isRoyalStraightFlash, Poker.isStraightFlash, Poker.isFourCard, Poker.isFullHouse, Poker.isFlash, Poker.isStraight, Poker.isThreeCard, Poker.isNPair):
            judge = judgement(self.player.hand)
            if not judge == Poker.Nothing:
                poker = judge
                break

        return poker, self.bids * SimplePokerEnv.reward_rate_map[poker]

    def _close(self):
        pass

    def _seed(self, seed=None):
        pass
    
    def _render(self, mode='human', close=False):
        outfile = StringIO() if mode == 'ansi' else sys.stdout
        outfile.write(self._observe() + '\n')
        return outfile

## Test code for each poker judgement 

- Reference: [遊びかた：ポーカー](https://www.nintendo.co.jp/others/playing_cards/howtoplay/poker/index.html)

### RoyalStraightFlash

In [59]:
from collections import OrderedDict
cards = OrderedDict()
cards["RoyalStraightFlash"] = [[Card(Suit.Spade,1), Card(Suit.Spade,10), Card(Suit.Spade,11), Card(Suit.Spade,12), Card(Suit.Spade,13), Card(Suit.Heart,1), Card(Suit.Diamond,7)]]
cards["StraightFlash"] = [[Card(Suit.Spade,9), Card(Suit.Spade,10), Card(Suit.Spade,11), Card(Suit.Spade,12), Card(Suit.Spade,13), Card(Suit.Heart,1), Card(Suit.Diamond,7)]]
cards["FourCard"] = [[Card(Suit.Spade,1), Card(Suit.Heart,1), Card(Suit.Diamond,1), Card(Suit.Clover,1), Card(Suit.Spade,13), Card(Suit.Heart,3), Card(Suit.Diamond,7)]]
cards["FullHouse"] = [[Card(Suit.Spade,1), Card(Suit.Heart,1), Card(Suit.Diamond,3), Card(Suit.Clover,3), Card(Suit.Spade,3), Card(Suit.Heart,1), Card(Suit.Diamond,10)]]
cards["Flash"] = [[Card(Suit.Spade,1), Card(Suit.Spade,3), Card(Suit.Spade,8), Card(Suit.Spade,10), Card(Suit.Spade,13), Card(Suit.Heart,10), Card(Suit.Diamond,13)]]
cards["Straight"] = [[Card(Suit.Spade,1), Card(Suit.Heart,2), Card(Suit.Clover,3), Card(Suit.Diamond,4), Card(Suit.Spade,5), Card(Suit.Heart,10), Card(Suit.Diamond,13)]]
cards["ThreeCard"] = [[Card(Suit.Spade,1), Card(Suit.Heart,1), Card(Suit.Clover,1), Card(Suit.Diamond,5), Card(Suit.Spade,12), Card(Suit.Clover,3), Card(Suit.Spade,4)]]
cards["TwoPair"] = [[Card(Suit.Spade,1), Card(Suit.Heart,1), Card(Suit.Clover,2), Card(Suit.Diamond,2), Card(Suit.Spade,12), Card(Suit.Clover,7), Card(Suit.Spade,9)]]
cards["OnePair"] = [[Card(Suit.Spade,1), Card(Suit.Clover,1), Card(Suit.Spade,3), Card(Suit.Spade,4), Card(Suit.Diamond,5), Card(Suit.Clover,7), Card(Suit.Spade,9)]]
cards["NoPoker"] = [[Card(Suit.Spade,1), Card(Suit.Clover,3), Card(Suit.Heart,5), Card(Suit.Diamond,7), Card(Suit.Spade,9), Card(Suit.Clover,2), Card(Suit.Heart,8)]]


for key in cards:
    print("Expected {}.".format(key))
    for i in range(len(cards[key])):
        c = cards[key][i]
        random.shuffle(c)
        print(" - judgement is {}.".format(Poker.getPoker(c[0:2], c[2:])))
    print()

Expected RoyalStraightFlash.
 - judgement is Poker.RoyalStraightFlash.

Expected StraightFlash.
 - judgement is Poker.StraightFlash.

Expected FourCard.
 - judgement is Poker.FourCard.

Expected FullHouse.
 - judgement is Poker.FullHouse.

Expected Flash.
 - judgement is Poker.Flash.

Expected Straight.
 - judgement is Poker.Straight.

Expected ThreeCard.
 - judgement is Poker.ThreeCard.

Expected TwoPair.
 - judgement is Poker.TwoPair.

Expected OnePair.
 - judgement is Poker.OnePair.

Expected NoPoker.
 - judgement is Poker.Nothing.



## Create Environment instance

In [21]:
env = SimplePokerEnv()

## Solve with Random policy

### Define choose action policy and choose drop card policy.

In [22]:
import random
def choose_drop_card_nums():
    n = random.randint(0,5)
    choosen = [0,0,0,0,0]
    for c in random.sample([0,1,2,3,4],n):
        choosen[c] = 1
    return choosen

def choose_action():
    actions = [SinglePlayPokerActions.DRAW, SinglePlayPokerActions.DOUBLE]
    return random.choice(actions)

### Let's try with Random policy

In [23]:
n_times = 100
env = SimplePokerEnv()
rewards = []
pokers = []
results = []

def judge(cards):
    poker = Poker.Nothing
    for judgement in (Poker.isRoyalStraightFlash, Poker.isStraightFlash, Poker.isFourCard, Poker.isFullHouse, Poker.isFlash, Poker.isStraight, Poker.isThreeCard, Poker.isNPair):
        judge = judgement(cards)
        if not judge == Poker.Nothing:
            poker = judge
            break
    return poker

for i in range(n_times):
    env._reset()
    done = False
    reward = 0
    info = None
    while not done:
        p_state = env._observe()
        action = choose_action()
        drop_cards = choose_drop_card_nums()
        a = UserAction(action, drop_cards)
        n_state, reward, done, info = env._step(a)
        if done:
            #print("*** {}th try is done***".format(i+1))
            #print("{}th try observation is: {}.".format(i+1, observation))
            #print("{}th try poker is: {}.".format(i+1, info["poker"]))
            #print("{}th try reward is: {}.".format(i+1, reward))
            line = []
            line.append(n_state.current_bid)
            for c in p_state.current_hand:
                line.append(c.get_serial())
            
            line.append(judge(p_state.current_hand).value)
            line.append(action.value)
            
            for bit in drop_cards:
                line.append(bit)
                
            for c in n_state.current_hand:
                line.append(c.get_serial())
                
            line.append(info["poker"].value)
            line.append(reward)
            
            results.append(line)
        #print(observation)
        #print(done)
    #rewards.append(reward)
    #pokers.append(info["poker"])
#print("finally, you gain {} coins.".format(sum(rewards)))

In [24]:
import pandas as pd
df = pd.DataFrame(results,columns=["amount_bid", "pre_card1", "pre_card2", "pre_card3", "pre_card4", "pre_card5", "current_poker", "action", "drop_card1", "drop_card2", "drop_card3", "drop_card4", "drop_card5", "result_card1", "result_card2", "result_card3", "result_card4", "result_card5", "result_poker", "reward"])

In [25]:
df.to_pickle('random-{}try.pickle'.format(n_times))

In [26]:
df.head()

,amount_bid,pre_card1,pre_card2,pre_card3,pre_card4,pre_card5,current_poker,action,drop_card1,drop_card2,drop_card3,drop_card4,drop_card5,result_card1,result_card2,result_card3,result_card4,result_card5,result_poker,reward
0,1,19,7,20,35,12,8,0,1,0,1,1,1,7,46,48,11,51,8,1
1,2,42,30,5,6,12,9,1,1,1,1,1,0,14,15,46,34,12,9,-2
2,2,28,30,46,9,26,9,1,0,1,1,0,1,28,3,31,9,10,9,-2
3,2,27,4,5,31,26,8,1,0,0,0,1,0,27,29,4,5,26,9,-2
4,2,17,18,23,49,11,8,1,1,1,1,1,1,41,16,21,34,51,8,2
